In [18]:
#https://github.com/Rufflewind/revad/blob/eb3978b3ccdfa8189f3ff59d1ecee71f51c33fd7/revad.py
import math

class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        # initialize to None, which means it’s not yet evaluated
        self.grad_value = None

    def __add__(self, other):
        z = Var(self.value + other.value)
        self.children.append((1.0, z))
        other.children.append((1.0, z))
        return z

    def __mul__(self, other):
        z = Var(self.value * other.value)
        self.children.append((other.value, z))
        other.children.append((self.value, z))
        return z
    
    def grad(self):
        # recurse only if the value is not yet cached
        if self.grad_value is None:
            # calculate derivative using chain rule
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value
# define the Vars for the example problem
# initialize x = 0.5 and y = 4.2
def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z

x = Var(0.5)
y = Var(4.2)
a= x*y

In [13]:
a.grad_value = 1.0
print("∂a/∂x = {}".format(x.grad())) # ∂a/∂x = 4.2
print("∂a/∂y = {}".format(y.grad())) 

∂a/∂x = 4.2
∂a/∂y = 0.5


In [15]:
print(a.value)

2.1


In [19]:
x = Var(0.5)
y = Var(4.2)
z= x*y+sin(x)
z.grad_value = 1.0

assert abs(z.value - 2.579425538604203) <= 1e-15
assert abs(x.grad() - (y.value + math.cos(x.value))) <= 1e-15
assert abs(y.grad() - x.value) <= 1e-15